In [1]:
import csv
from Font_classifier import Font_classifier
from download_h5py_db import download_file_from_google_drive
from tqdm import tqdm
from tensorflow.keras.models import load_model
from pathlib import Path
from utils import show_results




In [2]:
def load_classifier_model(path):
   
    return Font_classifier(model)


# def create_output():
#     with open('train_lables.csv', 'w+', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow([" ", "image", "char","Open Sans","Sansation","Titillium Web","Ubuntu Mono","Alex Brush"])
        


In [3]:
FILE_NAME = 'test_data.h5'
if not Path('test_data.h5').exists():
    download_file_from_google_drive('1YwLcXqLArFSOtoepQw7nC1t4jC8CFxpI', FILE_NAME)


In [4]:
path = 'res//top_model.h5'
model = load_model(path)


In [5]:
classifier = Font_classifier(model)

In [6]:
# print(db['data'].keys())

In [10]:
from sklearn.preprocessing import normalize
from utils import create_dirs, SIZE, get_image_data, crop_and_save, font_to_num
from numpy import asarray, argmax, bincount,argwhere
from h5py import File
from tqdm import tqdm
import numpy as np

In [23]:
from keras.utils import np_utils
np_utils.to_categorical(1, 5)

array([0., 1., 0., 0., 0.], dtype=float32)

In [21]:
import csv

In [76]:
def predict(model, db_file):
        db = File(db_file, 'r')   
        size=SIZE
        im_names = list(db['data'].keys())
        num = 0
        index = 0
        # prediction_arr=[]
        # images = []
        # chars=[]
        with open('test_lables.csv', 'w+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([" ", "image", "char","Open Sans","Sansation","Titillium Web","Ubuntu Mono","Alex Brush"])
            for i in tqdm(range(0, len(im_names))):
                im = im_names[i]
                img, _, txts, charBBs, wordBBs = get_image_data(db, im, no_labels=True)
                font_indx = 0 
                char_indx = 0
                for j in range(0, len(txts)):
                    test_x = [] 
                    cropped = crop_and_save(img, wordBBs, j, size, None, im, num, True)
                    test_x.append(cropped)
                    num+=1            
                    for k in range(0, len(txts[j])):
                        word =  txts[j]
                        cropped = crop_and_save(img, charBBs, char_indx, size, None, im, num, True)
                        test_x.append(cropped)
                        num+=1
                        # char_indx+=1
                        index += 1
                    test_x = asarray(test_x, dtype=np.float32)
                    reses = model.predict(test_x, verbose=0)
                    maxes = argmax(reses, axis=1)
                    prediction = bincount(maxes)
                    prediction = argwhere(prediction==prediction.max())
                    if (len(prediction)>1):
                        reses_n = normalize(reses, axis=1, norm='l1')
                        maxes_n = argmax(reses_n, axis=1)
                        prediction = bincount(maxes_n)
                        prediction = argwhere(prediction==prediction.max())
                        if(len(prediction)>1):
                            sum_p = np.sum(reses, axis=0)
                            prediction = sum_p.argmax()
                    for k in range(0, len(word)):
                        a = []
                        a.append(index)
                        a.append(im)
                        a.append(chr(word[k]))
                        b = np_utils.to_categorical(prediction.item(), 5)
                        b = np.concatenate((a,b[1:],[b[0]]))
                        # print(b)
                        writer.writerow(b)
                        index+=1
                        # prediction_arr.append(prediction.item())
                        # images.append(im)
                    font_indx += len(txts[j])
        # return prediction_arr

In [77]:
pred_y = predict(model, FILE_NAME)


100%|██████████| 1004/1004 [13:57<00:00,  1.20it/s]
